In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
import ctypes
from pycuda.compiler import SourceModule
import time
import faiss
print("good")

good


In [2]:

#define CLIP(x, min, max) (fminf(fmaxf(x, min), max))

N = 10  # Increase to capture more user preferences
  # Adjust as needed
totalMoviesUsingPerPerson=1000

# Define the Row structure
class Row(ctypes.Structure):
    _fields_ = [("col1", ctypes.c_int), ("col2", ctypes.c_float)]
class RowTest(ctypes.Structure):
    _fields_ = [("col1", ctypes.c_int), ("col2", ctypes.c_float),("user",ctypes.c_int)]



# Create some example row data
rows_data = []

# Define the parallel matrix multiplication kernel
kernel_code = """
struct Row {
    int col1;
    float col2;
};

__global__ void matmul(float *A, float *B, int N, int raters, int movies, Row *rows, float learning_rate) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;  // Movies
    int col = blockIdx.x * blockDim.x + threadIdx.x;  // Raters
    int totalMoviesUsingPerPerson=1000;
    if (row < movies && col < raters && col > 0 && row > 0) {
        int found = 0;
        float actual_rating = 0.0;

        // Find actual rating from stored data
        
        int start = 0;
        int end = totalMoviesUsingPerPerson-1;  // Fix here to prevent out-of-bounds access
        while (start <= end) {
            int mid = start + (end - start) / 2;
            if (rows[(col - 1) * totalMoviesUsingPerPerson + mid].col1 == row){
                found = 1;
                actual_rating = rows[(col - 1) * totalMoviesUsingPerPerson + mid].col2;
                break;
            } else if (rows[(col - 1) * totalMoviesUsingPerPerson + mid].col1==0 || rows[(col - 1) * totalMoviesUsingPerPerson + mid].col1 > row) {
                end = mid - 1;
            } else {
                start = mid + 1;
            }
        }

        if (found) {
            float predicted_rating = 0.0;
            for (int k = 0; k < N; k++) {
                predicted_rating += A[row * N + k] * B[k * raters + col];
            }
            
            float error = actual_rating - predicted_rating;
            
            for (int k = 0; k < N; k++) {
                float lambda = 0.01;
                float regularization = lambda * (A[row * N + k] * A[row * N + k] + B[k * raters + col] * B[k * raters + col]);
                float grad_A = -2.0 * error * B[k * raters + col] + regularization;
                float grad_B = -2.0 * error * A[row * N + k] + regularization;
                                
                // Apply updates
                atomicAdd(&A[row * N + k], -learning_rate * grad_A);
                atomicAdd(&B[k * raters + col], -learning_rate * grad_B);

                atomicExch(&A[row * N + k], fminf(fmaxf(A[row * N + k], -1.0f), 1.0f));
                atomicExch(&B[k * raters + col], fminf(fmaxf(B[k * raters + col], -1.0f), 1.0f));
            }

            

        }

    }
}

"""

mod = SourceModule(kernel_code)


In [3]:


def read_csv_to_list_rating(file_path):
    """Read a CSV file into a list of lists with specific column types."""
    with open(file_path, 'r', encoding='utf-8') as file:
        matrix = []
        header = file.readline().strip().split(',')
        for index,line in enumerate(file):
            values = line.strip().split(',')
            
            row = [int(values[0]), int(values[1]), float(values[2]), int(values[3])]
            matrix.append(row)
        return header, matrix

def read_csv_to_list_movies(file_path):
    """Read a CSV file into a list of lists with specific column types."""
    with open(file_path, 'r', encoding='utf-8') as file:
        matrix = []
        header = file.readline().strip().split(',')
        index=1
        matrix.append([0,"",""])
        for line in (file):
            values = line.strip().split(',')
            while index<int(values[0]):
                matrix.append([0,"",""])
                index+=1
                
            row = [int(values[0]), str(values[1]), str(values[-1])]
            matrix.append(row)
            index+=1
        return header, matrix
headerMovies, matrixMovies = read_csv_to_list_movies('movies.csv')
headerRating, matrixrating = read_csv_to_list_rating('ratings.csv')


totalRaters = int(matrixrating[-1][0])+1  # total rows
totalMovies = int(matrixMovies[-1][0])+1
print(totalRaters)
print(totalMovies)


d = {}
# row 0 is userID

########################################################################## I CHANGED HERE#####################
#
for row in matrixrating:
    if row[0] not in d:
        d[row[0]] = []
        d[row[0]].append(list(row[1:4]))
    else:
        d[row[0]].append(list(row[1:4]))

for key, val in d.items():
    d[key] = sorted(val, key=lambda x: x[2])

for key, val in d.items():
    d[key] = d[key][::-1]

newD = {}
for key, val in d.items():
    newD[key] = sorted(d[key][:totalMoviesUsingPerPerson], key=lambda x: x[0])

totalUsers = len(newD)
total_rat_us=0
rows_data = []
for i in range(1, (len(newD) + 1)):
    row = newD[i]
    for j in range(totalMoviesUsingPerPerson):
        if j < len(row):
            total_rat_us+=1
            rows_data.append(Row(row[j][0], row[j][1]))
        else:
            rows_data.append(Row(0, 0.0))
print("Total rating used ",end="")
print(total_rat_us)
matrixAMovies = np.zeros((totalMovies, N), dtype=np.float32)
matrixBUsers = np.zeros((N, totalRaters), dtype=np.float32)


# Fill matrices with random values in the range -0.01 to 0.01
matrixAMovies = np.random.uniform(-0.1, 0.1, size=(totalMovies, N)).astype(np.float32)
matrixBUsers = np.random.uniform(-0.1, 0.1, size=(N, totalRaters)).astype(np.float32)

matrixAMovies = np.ascontiguousarray(matrixAMovies, dtype=np.float32)
matrixBUsers = np.ascontiguousarray(matrixBUsers, dtype=np.float32)

A_gpu_matrixAMovies = cuda.mem_alloc(matrixAMovies.nbytes)
B_gpu_matrixBUsers = cuda.mem_alloc(matrixBUsers.nbytes)

if A_gpu_matrixAMovies is None or B_gpu_matrixBUsers is None:
    print("Damn")

rows = (Row * len(rows_data))(*rows_data)

# Allocate memory on the GPU for the rows
rows_gpu = cuda.mem_alloc(ctypes.sizeof(rows))
if rows_gpu is None:
    print("damn")

# Transfer the rows data to the GPU
cuda.memcpy_htod(rows_gpu, rows)

cuda.memcpy_htod(A_gpu_matrixAMovies, matrixAMovies)
cuda.memcpy_htod(B_gpu_matrixBUsers, matrixBUsers)


matmul = mod.get_function("matmul")

# Define grid and block sizes
block_size = (16, 16, 1)  # 16x16 threads per block
grid_size = ((totalRaters + block_size[1] - 1) // block_size[1], 
             (totalMovies + block_size[0] - 1) // block_size[0])


print(f"Matrix A shape: {matrixAMovies.shape}")
print(f"Matrix B shape: {matrixBUsers.shape}")

# Calculate the shared memory size
shared_mem_size = ctypes.sizeof(Row) * min(totalRaters * totalMoviesUsingPerPerson, block_size[0] * block_size[1])
# Run the parallel CUDA kernel
cuda.Context.synchronize()  # Ensure synchronization before execution]
print("A GPU Memory Address:", int(A_gpu_matrixAMovies))
print("B GPU Memory Address:", int(B_gpu_matrixBUsers))
print("Rows GPU Memory Address:", int(rows_gpu))

# start_time = time.time()
# matmul(A_gpu_matrixAMovies, B_gpu_matrixBUsers, np.int32(N), np.int32(totalRaters), np.int32(totalMovies), rows_gpu,learning_rate,
#        block=block_size, grid=grid_size, shared=shared_mem_size)
# cuda.Context.synchronize()  # Ensure synchronization after execution
# end_time = time.time()
# # Copy result back to CPU

# # 🚀 First, copy updated A and B back to CPU
# matrixAMovies_result = np.empty_like(matrixAMovies)
# matrixBUsers_result = np.empty_like(matrixBUsers)

# cuda.memcpy_dtoh(matrixAMovies_result, A_gpu_matrixAMovies)
# cuda.memcpy_dtoh(matrixBUsers_result, B_gpu_matrixBUsers)
# rows_result = np.zeros(len(rows_data), dtype=[('col1', np.int32), ('col2', np.float32)])
# cuda.memcpy_dtoh(rows_result, rows_gpu)




200949
292758
Total rating used 29783075
Matrix A shape: (292758, 10)
Matrix B shape: (10, 200949)
A GPU Memory Address: 140355605364736
B GPU Memory Address: 140355617947648
Rows GPU Memory Address: 140333627211776


In [4]:
dictMovieIDtoName={}
i=0
for row in matrixMovies:
    if row[0]==0:
        continue

    if int(row[0]) not in dictMovieIDtoName:
        dictMovieIDtoName[int(row[0])] = [row[1],row[2]]
    
print("done")

done


In [5]:
print(len(matrixMovies))

292758


In [6]:
epochs = 75  # Number of iterations

initial_lr = np.float32(0.001)  # Start slightly lower than 0.003
decay_factor = np.float32(0.95)   # Reduce by 10% every 5 epochs
decay_step = 5

min_lr = np.float32(0.0005)  # Ensure learning rate never goes too low
def normalize_embeddings(matrix):
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)  # Compute row-wise norm
    norms[norms == 0] = 1  # Prevent division by zero
    return matrix / norms
learning_rate = np.float32(0.001)
matrixAMovies_result=None
matrixBUsers_result=None
rows_result=None
for epoch in range(epochs):
    # learning_rate = np.float32(initial_lr * np.exp(-0.05 * epoch))
    if epoch==10:
        learning_rate = np.float32(0.0005)
    if epoch==40:
        learning_rate = np.float32(0.0001)
    # if epoch ==40:
    #     learning_rate = np.float32(0.00005)
    matrixAMovies_result = np.empty_like(matrixAMovies)
    matrixBUsers_result = np.empty_like(matrixBUsers)
    rows_result = np.zeros(len(rows_data), dtype=[('col1', np.int32), ('col2', np.float32)])
    start_time = time.time()
    # if epoch % 5 == 0:  # Every 5 epochs, reduce learning rate
    #     learning_rate *= 0.9  # Reduce by 10%

    matmul(A_gpu_matrixAMovies, B_gpu_matrixBUsers, np.int32(N), np.int32(totalRaters),
           np.int32(totalMovies), rows_gpu, learning_rate, block=block_size, grid=grid_size,
           shared=shared_mem_size)
    
    cuda.Context.synchronize()  # Wait for CUDA execution

    # Copy updated matrices and rows back to CPU
    cuda.memcpy_dtoh(matrixAMovies_result, A_gpu_matrixAMovies)
    cuda.memcpy_dtoh(matrixBUsers_result, B_gpu_matrixBUsers)
    cuda.memcpy_dtoh(rows_result, rows_gpu)


    if np.isnan(matrixAMovies_result).any() or np.isnan(matrixBUsers_result).any():
        print("Error: NaN detected in A or B matrix!")
        exit()

    if np.isinf(matrixAMovies_result).any() or np.isinf(matrixBUsers_result).any():
        print("Error: Inf detected in A or B matrix!")
        exit()
    if (epoch%10==0):
        mse = 0
        count = 0
        for i in range(len(rows_result)):
            if rows_result[i]['col1'] > 0:
                actual = rows_result[i]['col2']
                predicted = np.dot(matrixAMovies_result[rows_result[i]['col1'], :], matrixBUsers_result[:, i // totalMoviesUsingPerPerson])
                
                if np.isnan(predicted) or np.isinf(predicted):
                    print(f"Error in prediction: row {i}, actual {actual}, predicted {predicted}")
                    exit()
        
                mse += (actual - predicted) ** 2
                count += 1
        
        mse = mse / count if count > 0 else float('inf')  # Avoid division by zero
        
        
        print(f"Epoch {epoch+1}/{epochs}, MSE: {mse:.5f}")
        print(f"MSE: {mse:.5f}")
    print(f'epoch {epoch} done')






# Track execution time for CUDA



user_id = 2  # Example user (change as needed)
movie_id = rows_result[17]['col1']  # Get the movie ID from rows_result

predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}")







listUnused =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==0:
        listUnused.append([int(m[1]),int(m[2])])

listUnused2 =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==1:
        listUnused2.append([int(m[1]),int(m[2])])


for i in range(len(listUnused)):
    user_id = 2  # Example user (change as needed)
    movie_id = listUnused[i][0]  # Get the movie ID from rows_result
    actualPred = listUnused[i][1]
    predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
    print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}     Actual {actualPred}")

print()

Epoch 1/75, MSE: 11.43201
MSE: 11.43201
epoch 0 done
epoch 1 done
epoch 2 done
epoch 3 done
epoch 4 done
epoch 5 done
epoch 6 done
epoch 7 done
epoch 8 done
epoch 9 done
Epoch 11/75, MSE: 1.16420
MSE: 1.16420
epoch 10 done
epoch 11 done
epoch 12 done
epoch 13 done
epoch 14 done
epoch 15 done
epoch 16 done
epoch 17 done
epoch 18 done
epoch 19 done
Epoch 21/75, MSE: 1.15422
MSE: 1.15422
epoch 20 done
epoch 21 done
epoch 22 done
epoch 23 done
epoch 24 done
epoch 25 done
epoch 26 done
epoch 27 done
epoch 28 done
epoch 29 done
Epoch 31/75, MSE: 1.15878
MSE: 1.15878
epoch 30 done
epoch 31 done
epoch 32 done
epoch 33 done
epoch 34 done
epoch 35 done
epoch 36 done
epoch 37 done
epoch 38 done
epoch 39 done
Epoch 41/75, MSE: 1.16639
MSE: 1.16639
epoch 40 done
epoch 41 done
epoch 42 done
epoch 43 done
epoch 44 done
epoch 45 done
epoch 46 done
epoch 47 done
epoch 48 done
epoch 49 done
Epoch 51/75, MSE: 1.16840
MSE: 1.16840
epoch 50 done
epoch 51 done
epoch 52 done
epoch 53 done
epoch 54 done
epoch

In [ ]:
print("HERE")
print()
for i in range(len(listUnused2)):
    user_id = 2  # Example user (change as needed)
    movie_id = listUnused2[i][0]  # Get the movie ID from rows_result
    actualPred = listUnused2[i][1]
    predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
    print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}     Actual {actualPred}")

In [ ]:
print(rows_result)

In [ ]:
learning_rate = np.float32(0.001)

In [ ]:
listUnused =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==0:
        listUnused.append([int(m[1]),int(m[2])])

listUnused2 =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==1:
        listUnused2.append([int(m[1]),float(m[2])])


print("HERE")
print()
for i in range(len(listUnused2)):
    user_id = 2  # Example user (change as needed)
    movie_id = listUnused2[i][0]  # Get the movie ID from rows_result
    actualPred = listUnused2[i][1]
    predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
    print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}     Actual {actualPred}")

In [ ]:
dictTest={}

for key, val in d.items():
    dictTest[key] = sorted(d[key][totalMoviesUsingPerPerson:], key=lambda x: x[0])

In [ ]:
def findMovie(id,rating=None):
    if rating!=None:
        print(f"{id} Movie name is {matrixMovies[id]} Rated {rating}")
    else:
        print(f"{id} Movie name is {matrixMovies[id]}")

In [ ]:
v=0
z=0
for key,val in dictTest.items():
    v+=len(val)
    if len(val)==0:
        z+=1
print(v)
print(z)

In [ ]:
v=0
for key,val in newD.items():
    v+=len(val)
    
print(v)


In [ ]:
24704769+7094487

In [ ]:
print(len(matrixrating))

In [ ]:

mse = 0
count = 0
for i in range(len(rows_result)):
    if rows_result[i]['col1'] > 0:
        actual = rows_result[i]['col2']
        predicted = np.dot(matrixAMovies_result[rows_result[i]['col1'], :], matrixBUsers_result[:, i // totalMoviesUsingPerPerson])
        
        if np.isnan(predicted) or np.isinf(predicted):
            print(f"Error in prediction: row {i}, actual {actual}, predicted {predicted}")
            exit()

        mse += (actual - predicted) ** 2
        count += 1

mse = mse / count if count > 0 else float('inf')  # Avoid division by zero


print(f"Epoch {epoch+1}/{epochs}, MSE: {mse:.5f}")
print(f"MSE: {mse:.5f}")

KNN

In [ ]:
def normalize_embeddings(matrix):
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Avoid division by zero
    return matrix / norms

import numpy as np
import faiss
import time
# Start timing
start = time.time()
normalized_movies=matrixAMovies_result
# Build FAISS index
d = normalized_movies.shape[1]
index = faiss.IndexFlatIP(d)
index.add(normalized_movies)

# def compute_cosine_similarity(matrix):
#     return np.dot(matrix, matrix.T)

# similarity_matrix = compute_cosine_similarity(normalized_movies)
# Stop timing
end = time.time()

# Print timing info
print(f"FAISS-CPU indexing time: {end - start:.4f} seconds")


In [ ]:
import numpy as np
import faiss
import time

def normalize_embeddings(matrix):
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Avoid division by zero
    return matrix / norms

# Your movie embedding matrix
normalized_movies=matrixAMovies_result
d = normalized_movies.shape[1]

start = time.time()

#Set up FAISS-GPU
res = faiss.StandardGpuResources()
cpu_index = faiss.IndexFlatIP(d)
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

# Add data
gpu_index.add(normalized_movies)

end = time.time()

print(f"FAISS-GPU indexing time: {end - start:.4f} seconds")


In [ ]:
import numpy as np
import faiss

m = 1974  # ID of movie to query
#40815
#260
#1371
#1994
#1969
# Make sure the matrix is normalized (if not already)
normalized_movies = normalize_embeddings(matrixAMovies_result)

# Query vector
query_vector = normalized_movies[m].reshape(1, -1).astype('float32')  # FAISS expects float32 and 2D

# Setup FAISS-GPU index
d = normalized_movies.shape[1]
res = faiss.StandardGpuResources()
cpu_index = faiss.IndexFlatIP(d)
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

# Add data to GPU
gpu_index.add(normalized_movies)

# Do the search
k = 10
D, I = gpu_index.search(query_vector, k)

# Output results
print("Top", k, "similar movies to movie:")
findMovie(m)
print()
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    findMovie(idx)


In [ ]:
m=1974
#40815
#260
#1371
#1994
#1969


# Movie ID 1 is at row index 1 (if 0-based index)
query_vector = normalized_movies[m].reshape(1, -1)  # FAISS expects (batch_size, dim)

k = 10  # Number of similar movies you want
D, I = index.search(query_vector, k)

print("Top", k, "similar movies to movie:")
findMovie(m)
print()
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    # print(f"{rank + 1}: Movie ID {idx} with similarity {similarity:.4f}")
    findMovie(idx)


In [ ]:
def get_top_k_similar_movies_single(movie_id, normalized_movies, k=5):
    movie_vector = normalized_movies[movie_id]
    similarities = np.dot(normalized_movies, movie_vector)  # Only one movie
    top_k_indices = np.argsort(similarities)[-k-1:-1][::-1]  # Exclude itself
    return top_k_indices

# Usage:
top_k = get_top_k_similar_movies_single(m, normalized_movies=normalized_movies, k=5)
print(top_k)
    

In [ ]:


findMovie(123)

In [ ]:
for id in top_k:
    findMovie(id)

In [ ]:

# Search for similar movies
movie_id = 72
k = 5
query_vector = normalized_movies[movie_id:movie_id+1]

distances, indices = index.search(query_vector, k)


findMovie(movie_id)
print(f"Top {k} similar movies to movie {movie_id} are: {indices.flatten()}")
print()

for id in indices.flatten():
    print(id,end=" ")
    findMovie(id)

    

In [ ]:
d2 = matrixBUsers_result.shape[1]
index = faiss.IndexFlatIP(d2)
index.add(matrixBUsers_result)

In [ ]:
# Movie ID 1 is at row index 1 (if 0-based index)
userID=1

query_vector = matrixBUsers_result[userID].reshape(1, -1)  # FAISS expects (batch_size, dim)

k = 10  # Number of similar movies you want
D, I = index.search(query_vector, k)

print("Top", k, "similar movies to movie ID",userID,":")
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    print(f"{rank + 1}: User ID {idx} with similarity {similarity:.4f}")


In [ ]:
user_embeddings = matrixBUsers_result.T.copy(order="C")  # Make sure it's C-contiguous

# Build the FAISS index
d = user_embeddings.shape[1]  # Dimensionality (N)
index = faiss.IndexFlatIP(d)  # Inner product (good for cosine similarity)
index.add(user_embeddings)  # Add user vectors

# --- QUERY PART ---
userID = 1  # User ID you want neighbors for

query_vector = user_embeddings[userID].reshape(1, -1).astype(np.float32)  # FAISS expects (batch_size, dim)

k = 10  # Number of similar users you want
D, I = index.search(query_vector, k)

print(f"Top {k} similar users to user ID {userID}:")
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    print(f"{rank + 1}: User ID {idx} with similarity {similarity:.4f}")

In [ ]:
for i in range(100):
    if rows_data[i]==None:
        break
    if rows_data[i].col2<4.4:
        continue
    print(dictMovieIDtoName[rows_data[i].col1],rows_data[i].col2)
    

In [ ]:
for i in range(len(I[0])//2):
    idx=I[0][i]
    print(f"USER ID {idx}")
    for j in range(100):
            if rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col1==0:
                  break
            if (rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col2<4.4):
                  continue
            findMovie(rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col1,rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col2)
            

In [ ]:
for i in range(totalMoviesUsingPerPerson):
    print(rows_data[(45-1)*totalMoviesUsingPerPerson+i].col1)

This is to show that the gradients from the matrix Multiplication were able to find similar patterns in Movies. With random gradients it cannot find any patterns. With our updated gradients it finds similar movies.

In [ ]:
###########################################TEST####################

matrixAMoviesTEST = np.random.uniform(-0.1, 0.1, size=(totalMovies, N*100)).astype(np.float32)
matrixBUsersTEST = np.random.uniform(-0.1, 0.1, size=(N*100, totalRaters)).astype(np.float32)

In [ ]:
# Assume matrixAMovies_result is (totalMovies, N)

# 1. Build the FAISS index
d = matrixAMoviesTEST.shape[1]  # Dimensionality (N)
index = faiss.IndexFlatIP(d)  # IP = Inner Product (cosine similarity if vectors normalized)
index.add(matrixAMoviesTEST)  # Add all movie embeddings to index

# 2. Query the FAISS index
movie_id = 40815  # Some movie ID you want neighbors for
query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)

k = 5  # Number of neighbors
D, I = index.search(query, k)

print("Top K similar movie IDs:", I)
print("Similarities:", D)
print("Top", k, "similar movies to movie:")
findMovie(40815)
print()
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    # print(f"{rank + 1}: Movie ID {idx} with similarity {similarity:.4f}")
    findMovie(idx)

In [ ]:
matrixAMoviesTEST = np.random.uniform(-0.1, 0.1, size=(292758, 50)).astype(np.float32)

In [ ]:


start = time.time()
print("start")
print(matrixAMoviesTEST.shape)
#Set up FAISS
d = matrixAMoviesTEST.shape[1]  # Dimensionality (N)
index = faiss.IndexFlatIP(d)  # IP = Inner Product (cosine similarity if vectors normalized)
index.add(matrixAMoviesTEST)  # Add all movie embeddings to index

# 2. Query the FAISS index
movie_id = 40815  # Some movie ID you want neighbors for
query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)

k = 5  # Number of neighbors
D, I = index.search(query, k)



end = time.time()

print(f"FAISS-CPU indexing time: {end - start:.4f} seconds")

In [ ]:
import numpy as np
import faiss
import time

# Generate random test matrices
print(matrixAMoviesTEST.shape)
start = time.time()

# Set up FAISS-GPU
d = matrixAMoviesTEST.shape[1]
res = faiss.StandardGpuResources()  # GPU resource manager
cpu_index = faiss.IndexFlatIP(d)
gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)

# Add embeddings to GPU index
gpu_index.add(matrixAMoviesTEST)

# Query the index
movie_id = 40815
query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)

k = 5
D, I = gpu_index.search(query, k)




end = time.time()
print(f"FAISS-GPU total time: {end - start:.4f} seconds")


In [ ]:
import numpy as np
import faiss
import time
li = [10,50,100,200,300,400]
for i in li:
    matrixAMoviesTEST = np.random.uniform(-0.1, 0.1, size=(292758*i, 64)).astype(np.float32)
    
    start = time.time()
    print("start")
    print(matrixAMoviesTEST.shape)
    #Set up FAISS
    d = matrixAMoviesTEST.shape[1]  # Dimensionality (N)
    index = faiss.IndexFlatIP(d)  # IP = Inner Product (cosine similarity if vectors normalized)
    index.add(matrixAMoviesTEST)  # Add all movie embeddings to index
    
    # 2. Query the FAISS index
    movie_id = 4  # Some movie ID you want neighbors for
    query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)
    
    k = 5  # Number of neighbors
    D, I = index.search(query, k)
    
    
    
    end = time.time()
    
    print(f"FAISS-CPU indexing train time: {end - start:.4f} seconds")
    
    start = time.time()
    movie_id = 8  # Some movie ID you want neighbors for
    query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)
    
    k = 5  # Number of neighbors
    D, I = index.search(query, k)
    end = time.time()
    
    print(f"FAISS-CPU indexing search time: {end - start:.4f} seconds")
    
    # Generate random test matrices
    print(matrixAMoviesTEST.shape)
    start = time.time()
    
    # Set up FAISS-GPU
    d = matrixAMoviesTEST.shape[1]
    res = faiss.StandardGpuResources()  # GPU resource manager
    cpu_index = faiss.IndexFlatIP(d)
    gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
    
    # Add embeddings to GPU index
    gpu_index.add(matrixAMoviesTEST)
    end = time.time()
    print(f"FAISS-GPU train time: {end - start:.4f} seconds")
    start = time.time()
    # Query the index
    movie_id = 40815
    query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)
    
    k = 5
    D, I = gpu_index.search(query, k)
    end = time.time()
    print(f"FAISS-GPU search time: {end - start:.4f} seconds")
    
    
    

In [ ]:
import numpy as np
import faiss
import time
li = [50,100,250,500,1000]
for i in li:
    # Simulate a large movie matrix
    matrixAMoviesTEST = np.random.uniform(-0.1, 0.1, size=(292758*1000, 64)).astype(np.float32)
    d = matrixAMoviesTEST.shape[1]  # d = 50
    nlist = 100  # Number of coarse clusters
    m =  8      # Number of subquantizers (must divide d)
    nbits = 8    # Bits per subvector
    batch_size = 500_000  # Add in chunks to avoid GPU OOM
    
    start = time.time()
    
    # 1. Build the coarse quantizer and IVF+PQ index on CPU
    quantizer = faiss.IndexFlatIP(d)
    cpu_index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)
    
    # 2. Train the index with a small subset
    np.random.seed(42)
    train_sample = matrixAMoviesTEST[np.random.choice(matrixAMoviesTEST.shape[0], 10_000, replace=False)]
    cpu_index.train(train_sample)
    
    # 3. Move to GPU
    res = faiss.StandardGpuResources()
    gpu_index = faiss.index_cpu_to_gpu(res, 0, cpu_index)
    
    # 4. Add data in chunks
    for i in range(0, matrixAMoviesTEST.shape[0], batch_size):
        chunk = matrixAMoviesTEST[i:i + batch_size]
        gpu_index.add(chunk)
    
    
    
    end = time.time()
    print(f"FAISS-GPU total train time: {end - start:.4f} seconds")
    start = time.time()
    # 5. Search
    query_vector = matrixAMoviesTEST[4].reshape(1, -1)
    gpu_index.nprobe = 10  # Number of clusters to search
    
    k = 5
    D, I = gpu_index.search(query_vector, k)
    
    end = time.time()
    print(f"FAISS-GPU total search time: {end - start:.4f} seconds")
